In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
train_data=pd.read_csv('train.csv', sep=',')
test_data=pd.read_csv('test.csv', sep=',')

FileNotFoundError: ignored

In [ ]:
train_data.head()


In [ ]:
test_data.head()

In [ ]:
print(train_data.shape)
print(test_data.shape)

In [ ]:
print(test_data.columns)
print(train_data.columns)

In [ ]:
#sns.heatmap(train_data.isnull(), cmap='Blues')

In [ ]:
#sns.heatmap(test_data.isnull(), cmap='Blues')

In [ ]:
print(train_data.reassignment_method.unique())
print(train_data.reassignment_reason.unique())
print(train_data.reassigned_order.unique())

In [ ]:
train_data[(train_data['reassigned_order']==1)].cancelled.unique()

In [ ]:
train_data.rider_id.unique()

In [ ]:
y=train_data.cancelled
X=train_data.drop(columns=['order_id','rider_id','reassignment_method', 'reassignment_reason', 'reassigned_order', 'pickup_time', 'delivered_time', 'cancelled', 'cancelled_time'], axis=1)
X_test=test_data.drop(columns=['order_id','rider_id', 'reassignment_method', 'reassignment_reason', 'reassigned_order'], axis=1)

In [ ]:
X=X.fillna(0)
X_test=X_test.fillna(0)

In [ ]:
X['month']=pd.DatetimeIndex(X['order_time']).month
X['day_of_week']=pd.DatetimeIndex(X['order_time']).day_of_week
X['order_hour']=pd.DatetimeIndex(X['order_time']).hour
X['order_minutes']=pd.DatetimeIndex(X['order_time']).minute
X['accept_hour']=pd.DatetimeIndex(X['accept_time']).hour
X['accept_minutes']=pd.DatetimeIndex(X['accept_time']).minute
X['order_seconds']=pd.DatetimeIndex(X['order_time']).second
X['accept_seconds']=pd.DatetimeIndex(X['accept_time']).second
X['process_time']=np.abs(np.abs((X['accept_hour']-X['order_hour'])*60)-(np.abs(X['accept_minutes']-X['order_minutes']))*60+np.abs(X['accept_seconds']-X['order_seconds']))
X['total_distance']=X['first_mile_distance']+X['last_mile_distance']
#X['rider_score']=X['delivered_orders']/(X['delivered_orders']+X['undelivered_orders'])
X['rider_like']=X['alloted_orders']/X['alloted_orders'].sum()

In [ ]:
X_test['month']=pd.DatetimeIndex(X_test['order_time']).month
X_test['day_of_week']=pd.DatetimeIndex(X_test['order_time']).day_of_week
X_test['order_hour']=pd.DatetimeIndex(X_test['order_time']).hour
X_test['order_minutes']=pd.DatetimeIndex(X_test['order_time']).minute
X_test['accept_hour']=pd.DatetimeIndex(X_test['accept_time']).hour
X_test['accept_minutes']=pd.DatetimeIndex(X_test['accept_time']).minute
X_test['order_seconds']=pd.DatetimeIndex(X_test['order_time']).second
X_test['accept_seconds']=pd.DatetimeIndex(X_test['accept_time']).second
X_test['process_time']=np.abs(np.abs((X_test['accept_hour']-X_test['order_hour'])*60)-(np.abs(X['accept_minutes']-X['order_minutes']))*60+ np.abs(X_test['accept_seconds']-X_test['order_seconds']))
X_test['total_distance']=X_test['first_mile_distance']+X_test['last_mile_distance']
#X_test['rider_score']=X_test['delivered_orders']/(X_test['delivered_orders']+X_test['undelivered_orders'])
X_test['rider_like']=X_test['alloted_orders']/X_test['alloted_orders'].sum()

In [ ]:
X=X.drop(columns=[ 'order_time', 'order_date', 'allot_time','accept_time', 'order_minutes','accept_hour', 'accept_minutes', 'first_mile_distance', 'last_mile_distance', 'order_time', 'delivered_orders', 'undelivered_orders', 'alloted_orders'], axis=1)
X_test=X_test.drop(columns=[ 'order_time', 'order_date',  'allot_time','accept_time', 'order_minutes','accept_hour', 'accept_minutes', 'first_mile_distance', 'last_mile_distance', 'order_time', 'delivered_orders', 'undelivered_orders', 'alloted_orders'], axis=1)

In [ ]:
X=X.fillna(0)
X_test=X_test.fillna(0)

In [ ]:
X.describe()

In [ ]:
from sklearn.model_selection import KFold, cross_val_score, train_test_split, RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler, PowerTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.initializers import Constant
from tensorflow.keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier

In [ ]:
cols=[ 'lifetime_order_count', 'session_time', 'process_time']

In [ ]:
print(X.shape)
print(X_test.shape)

In [ ]:
minmax=StandardScaler()
X=minmax.fit_transform(X)
X_test=minmax.fit_transform(X_test)

In [ ]:
fold_metrics=[]

In [ ]:
zero, one=np.bincount(y)
print(100*one/(one+zero))

In [ ]:
met=[keras.metrics.AUC(name='auc'),
     keras.metrics.BinaryAccuracy(name='accuracy')]

In [ ]:
def make_model(metrics=met, output_bias=None):
  if output_bias is not None:
    output_bias = Constant(output_bias)
    opt=keras.optimizers.Adam(learning_rate=0.01)
    model = Sequential()
    model.add(Dense(16, activation='relu', input_dim=X.shape[-1]))
    model.add(Dropout(0.5))
    model.add(Dense(8, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid', bias_initializer=output_bias))

  model.compile(optimizer=opt, loss='binary_crossentropy', metrics=met)

  return model

In [ ]:
bias=np.log([one/zero])
early_stopping=EarlyStopping(patience=50)

In [ ]:
model=make_model(output_bias=bias)
model.fit(X, y, epochs=100, batch_size=3000, validation_split=0.22)


In [ ]:
weight_0=(1/zero)*((zero+one)/2)
weight_1=(1/one)*((zero+one)/2)

class_weight = {0: weight_0, 1: weight_1}

In [ ]:
test_data['pred']=model.predict(X_test)

In [ ]:
test_data[test_data['pred']==1]['pred'].sum()

In [ ]:
res = pd.DataFrame(test_data, columns=['order_id', 'pred'])
res.columns = ['order_id','cancelled']

# To download the csv file locally
res.to_csv('submission.csv', index=False)  
from google.colab import files
files.download('submission.csv')